In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastai.vision.all import *
import sklearn.metrics as skm
from tqdm.notebook import tqdm
import sklearn.feature_extraction.text
from transformers import (BertTokenizer, BertModel,
                          AutoConfig, AutoModel)
import gc
import codecs

In [3]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [4]:
from shopee_utils import *

In [5]:
OUTPUT_CLASSES=11014
TAKE_PAIRS = 5.7

In [6]:
TRIAL_RUN=False

In [7]:
def efficientnet_b0(pretrained): return timm.create_model('efficientnet_b0', pretrained=pretrained)
def efficientnet_b1(pretrained): return timm.create_model('efficientnet_b1', pretrained=pretrained)
def efficientnet_b2(pretrained): return timm.create_model('efficientnet_b2', pretrained=pretrained)

In [8]:
class conf():
    bs = 64
    arch = efficientnet_b0
    arcface_m=.4
    arcface_s=30

In [9]:
text_model_name='bert_large_state0.pth'
img_model_name='b0_788.pth'

PATH = Path('../input/shopee-product-matching')
models_path = Path('../input/shopee-models')
BERT_PATH = '../input/bertlarge-config'

if not PATH.is_dir():
    PATH = Path('/home/slex/data/shopee')
    models_path = Path('./models')
    BERT_PATH = './indobert-large-p2'
image_model_file = models_path / img_model_name
text_model_file = models_path / text_model_name

In [10]:

train_df = pd.read_csv(PATH/'train.csv')
train_df = add_splits(train_df)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


## TEXT

In [11]:
class EmbsModel(nn.Module):
    def __init__(self, bert_model):
        super().__init__()
        self.bert_model = bert_model
    def forward(self, x):
        output = self.bert_model(*x)
        return output.last_hidden_state[:,0,:]

In [12]:
class TitleTransform(Transform):
    def __init__(self):
        super().__init__()
        self.tokenizer = BertTokenizer.from_pretrained(BERT_PATH)
        
        
    def encodes(self, row):
        text = row.title
        text=(codecs.decode(text, 'unicode_escape'))
        encodings = self.tokenizer(text, padding = 'max_length', max_length=50, truncation=True,return_tensors='pt')
        keys =['input_ids', 'attention_mask', 'token_type_ids'] 
        return tuple(encodings[key].squeeze() for key in keys)

In [13]:
tfm = TitleTransform()

data_block = DataBlock(
    blocks = (TransformBlock(type_tfms=tfm), 
              CategoryBlock(vocab=train_df.label_group.to_list())),
    splitter=ColSplitter(),
    get_y=ColReader('label_group'),
    )
text_dls = data_block.dataloaders(train_df, bs=256)


In [14]:
def generate_text_embs(dl):
    model = AutoModel.from_config(AutoConfig.from_pretrained(BERT_PATH))
    state = torch.load(text_model_file)
    model.load_state_dict(state)
    model = EmbsModel(model).cuda().eval()
    embs, ys = embs_from_model(model, dl)
    return embs, ys

def generate_text_pairs(dl):
    embs, ys = generate_text_embs(dl)
    target_matrix = ys[:,None]==ys[None,:]
    groups = [torch.where(t)[0].tolist() for t in target_matrix]
    dists, inds = get_nearest(embs, do_chunk(embs))
    pairs = sorted_pairs(dists, inds)[:len(embs)*10]
    return pairs, groups

In [15]:
# pairs, groups = generate_text_pairs(text_dls.valid)
# _=build_from_pairs(pairs, groups, True)

## IMAGE


In [16]:
class ArcFaceClassifier(nn.Module):
    def __init__(self, in_features, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(in_features, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        return x_norm @ W_norm
    


class ResnetArcFace(nn.Module):
    def __init__(self):
        super().__init__()
        self.body = create_body(conf.arch, cut=-2, pretrained=False)
        nf = num_features_model(nn.Sequential(*self.body.children()))
        self.after_conv=nn.Sequential(
            AdaptiveConcatPool2d(),
            Flatten(),
            nn.BatchNorm1d(nf*2),
            nn.Dropout(.25))
        self.classifier = ArcFaceClassifier(nf*2, OUTPUT_CLASSES)
        self.outputEmbs = False
    def forward(self, x):
        x = self.body(x)
        embeddings = self.after_conv(x)
        if self.outputEmbs:
            return embeddings
        return self.classifier(embeddings)

In [17]:
def get_img_file(row):
    img =row.image
    fn  = PATH/'train_images'/img
    if not fn.is_file():
        fn = PATH/'test_images'/img
    return fn

In [18]:
data_block = DataBlock(blocks = (ImageBlock(), CategoryBlock(vocab=train_df.label_group.to_list())),
                 splitter=ColSplitter(),
                 get_y=ColReader('label_group'),
                 get_x=get_img_file,
                 item_tfms=Resize(460),
                 batch_tfms=aug_transforms(size=224, min_scale=0.75),
                 )
dls_image = data_block.dataloaders(train_df, bs=conf.bs)


In [19]:
def generate_image_embs(dl):
    model = ResnetArcFace()
    state = torch.load(image_model_file)
    model.load_state_dict(state['model'])
    model = model.cuda().eval()
    model.outputEmbs = True
    embs, ys = embs_from_model(model, dl)
    return embs,ys

def generate_image_pairs(dl):
    embs, ys = generate_image_embs(dl)
    target_matrix = ys[:,None]==ys[None,:]
    groups = [torch.where(t)[0].tolist() for t in target_matrix]
    dists, inds = get_nearest(embs, do_chunk(embs))
    pairs = sorted_pairs(dists, inds)[:len(embs)*10]
    return pairs, groups

In [20]:
# pairs, groups = generate_image_pairs(dls_image.valid)
# _=build_from_pairs(pairs, groups, True)

## Distances for combined top-K indices

In [21]:
def combined_dists(img_embs, text_embs):
    _, img_inds = get_nearest(img_embs, do_chunk(img_embs))

    _, text_inds = get_nearest(text_embs, do_chunk(text_embs))

    combined_inds = torch.cat([img_inds, text_inds],dim=1)
    img_dists = get_dist_for_inds(img_embs, combined_inds)

    text_dists = get_dist_for_inds(text_embs, combined_inds)
    return combined_inds, img_dists, text_dists

## Check on validation set

In [22]:
if TRIAL_RUN:
    img_embs,ys = generate_image_embs(dls_image.valid)

    text_embs,ys = generate_text_embs(text_dls.valid)

    target_matrix = ys[:,None]==ys[None,:]
    targets = [torch.where(t)[0].tolist() for t in target_matrix]

    combined_inds, img_dists, text_dists = combined_dists(img_embs, text_embs)
    
    # --- use max
    max_dists = torch.max(img_dists, text_dists)
    triplets = sorted_pairs(max_dists, combined_inds)[:len(img_embs)*10]
    _=build_from_pairs(triplets, targets)
    # --- use and
    and_dists = img_dists+text_dists - img_dists*text_dists
    triplets = sorted_pairs(and_dists, combined_inds)[:len(img_embs)*10]
    _=build_from_pairs(triplets, targets)
    # --- used scaled and
    img_params={'min_val': 0.26708984375, 'a': -3.8065830142124857, 'b': 4.92549411772561, 'c': 1.031201433949806}
    text_params={'min_val': 0.68115234375, 'a': -785.4151875846186, 'b': 9.68828642715568, 'c': 1.0604532834375875}
    img_scaler = functools.partial(do_scale, **img_params)
    text_scaler = functools.partial(do_scale, **text_params)

    scaled_img = img_scaler(img_dists)
    scaled_txt = text_scaler(text_dists)
    and_scaled = scaled_txt+scaled_img - scaled_txt*scaled_img
    triplets = sorted_pairs(and_scaled, combined_inds)[:len(img_embs)*10]
    _=build_from_pairs(triplets, targets)

In [23]:
if TRIAL_RUN:
    groups =group_and_shave(and_dists, combined_inds, train_df)
    groups_lens = [len(g)for g in groups]
    targets_lens = [len(g) for g in targets]
    plt.figure(figsize=(8,8)) 
    plt.hist((groups_lens,targets_lens) ,bins=list(range(2,52)))
    print(score_all_groups(groups, targets))

## Run on the test set

In [24]:
test_df = pd.read_csv(PATH/'test.csv')

In [25]:
if TRIAL_RUN:
    fake_test_df = train_df[['posting_id', 'image', 'image_phash', 'title', 'label_group']].copy()
    fake_test_df = pd.concat([fake_test_df, fake_test_df])
    fake_test_df = add_target_groups(fake_test_df)
    test_df = fake_test_df

In [26]:
text_embs,_ = generate_text_embs(text_dls.test_dl(test_df))

  0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
gc.collect()
torch.cuda.empty_cache()

In [28]:
test_dl = dls_image.test_dl(test_df)
img_embs,_ = generate_image_embs(test_dl)

  0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
combined_inds, img_dists, text_dists = combined_dists(img_embs, text_embs)

In [30]:
# img_params={'min_val': 0.27685546875, 'a': -3.2861138727246146, 'b': 4.026080848765254, 'c': 1.0652386207714422}
# text_params={'min_val': 0.4521484375, 'a': -3.7661719860419227, 'b': 2.1991546248531804, 'c': 1.413136700324773}
# img_scaler = functools.partial(do_scale, **img_params)
# text_scaler = functools.partial(do_scale, **text_params)

# scaled_img = img_scaler(img_dists)
# scaled_txt = text_scaler(text_dists)
# and_scaled = scaled_txt+scaled_img - scaled_txt*scaled_img
# combined = sorted_pairs(and_scaled, combined_inds)[:int(len(test_df)*TAKE_PAIRS)]

In [31]:
and_dists = img_dists+text_dists - img_dists*text_dists


In [32]:
groups =group_and_shave(and_dists, combined_inds, train_df)

In [33]:
if 'target' in test_df.columns.to_list():

    _=build_from_pairs(combined, test_df.target.to_list())
   

In [34]:
#combined = sorted_pairs(and_dists, combined_inds)[:int(len(test_df)*TAKE_PAIRS)]
# groups = [[] for _ in range(len(test_df))]
# for x,y,v in combined:
#     groups[x].append(y)

In [35]:
matches = [' '.join(test_df.iloc[g].posting_id.to_list()) for g in groups]
test_df['matches'] = matches

test_df[['posting_id','matches']].to_csv('submission.csv',index=False)

In [36]:
pd.read_csv('submission.csv').head()

,posting_id,matches
0,test_2255846744,test_2255846744 test_3588702337
1,test_3588702337,test_3588702337 test_4015706929 test_2255846744
2,test_4015706929,test_4015706929 test_3588702337 test_2255846744
